In [73]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
from puzzles import load, load_lines

In [75]:
def swp(s, i, j):
    i, j = min(i, j), max(i, j)
    return "".join([s[:i], s[j], s[i + 1 : j], s[i], s[j + 1 :]])


def swl(s, c1, c2):
    return swp(s, s.index(c1), s.index(c2))


def rot(s, n, to_right):
    n %= len(s)
    if to_right:
        return s[-n:] + s[:-n]
    return s[n:] + s[:n]


def rotb(s, c):
    x = s.index(c)
    return rot(s, x + 2 if x >= 4 else x + 1, to_right=True)


def rev(s, i, j):
    i, j = min(i, j), max(i, j)
    return "".join([s[:i], s[i : j + 1][::-1], s[j + 1 :]])


def move(s, i, j):
    c = s[i]
    ss = "".join([s[:i], s[i + 1 :]])
    return "".join([ss[:j], c, ss[j:]])

In [76]:
from tqdm import tqdm

In [77]:
lines = load_lines(21)
s = "abcdefgh"

for cmd in tqdm(lines):
    words = cmd.split()

    if cmd.startswith("swap position"):
        s = swp(s, int(words[2]), int(words[-1]))
    elif cmd.startswith("swap letter"):
        s = swl(s, words[2], words[-1])
    elif cmd.startswith("rotate based"):
        s = rotb(s, words[-1])
    elif cmd.startswith("rotate"):
        s = rot(s, int(words[2]), to_right=words[1] == "right")
    elif cmd.startswith("reverse"):
        s = rev(s, int(words[2]), int(words[-1]))
    elif cmd.startswith("move"):
        s = move(s, int(words[2]), int(words[-1]))
    else:
        raise
s

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 418175.87it/s]


'baecdfgh'

---

In [78]:
# def rotb(s, c):
#     x = s.index(c)
#     return rot(s, x + 1 if x >= 4 else x + 2, to_right=False)


def unrotb(s, c):
    for i in range(len(s)):
        rotated = rot(s, i, to_right=False)
        if rotb(rotated, c) == s:
            return rotated

In [79]:
lines = load_lines(21)
s = "fbgdceah"

for cmd in tqdm(lines[::-1]):
    words = cmd.split()

    if cmd.startswith("swap position"):
        s = swp(s, int(words[2]), int(words[-1]))
    elif cmd.startswith("swap letter"):
        s = swl(s, words[2], words[-1])
    elif cmd.startswith("reverse"):
        s = rev(s, int(words[2]), int(words[-1]))
    elif cmd.startswith("rotate based"):
        s = unrotb(s, words[-1])
    elif cmd.startswith("rotate"):
        s = rot(s, int(words[2]), to_right=words[1] != "right")
    elif cmd.startswith("move"):
        s = move(s, int(words[-1]), int(words[2]))
    else:
        raise
s

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 434642.90it/s]


'cegdahbf'

NOT THESE:
- efbchagd
- hfdacgeb

---

In [55]:
s = "abcde"

lines = [
    "swap position 4 with position 0",
    "swap letter d with letter b",
    "reverse positions 0 through 4",
    "rotate left 1 step",
    "move position 1 to position 4",
    "move position 3 to position 0",
    "rotate based on position of letter b",
    "rotate based on position of letter d",
]
answers = [
    "ebcda",
    "edcba",
    "abcde",
    "bcdea",
    "bdeac",
    "abdec",
    "ecabd",
    "decab",
]

for cmd, ans in zip(lines, answers):
    print(cmd.ljust(40), "| ", end="")
    words = cmd.split()

    if cmd.startswith("swap position"):
        s = swp(s, int(words[2]), int(words[-1]))
    elif cmd.startswith("swap letter"):
        s = swl(s, words[2], words[-1])
    elif cmd.startswith("rotate based"):
        s = rotb(s, words[-1])
    elif cmd.startswith("rotate"):
        s = rot(s, int(words[2]), to_right=words[1] == "right")
    elif cmd.startswith("reverse"):
        s = rev(s, int(words[2]), int(words[-1]))
    elif cmd.startswith("move"):
        s = move(s, int(words[2]), int(words[-1]))

    print(s, ans, "✅" if s == ans else "❌")

swap position 4 with position 0          | ebcda ebcda ✅
swap letter d with letter b              | edcba edcba ✅
reverse positions 0 through 4            | abcde abcde ✅
rotate left 1 step                       | bcdea bcdea ✅
move position 1 to position 4            | bdeac bdeac ✅
move position 3 to position 0            | abdec abdec ✅
rotate based on position of letter b     | ecabd ecabd ✅
rotate based on position of letter d     | decab decab ✅


- `swap position 4 with position 0` swaps the first and last letters, producing the input for the next step, ebcda.
- `swap letter d with letter b` swaps the positions of d and b: edcba.
- `reverse positions 0 through 4` causes the entire string to be reversed, producing abcde.
- `rotate left 1 step` shifts all letters left one position, causing the first letter to wrap to the end of the string: bcdea.
- `move position 1 to position 4` removes the letter at position 1 (c), then inserts it at position 4 (the end of the string): bdeac.
- `move position 3 to position 0` removes the letter at position 3 (a), then inserts it at position 0 (the front of the string): abdec.
- `rotate based on position of letter b` finds the index of letter b (1), then rotates the string right once plus a number of times equal to that index (2): ecabd.
- `rotate based on position of letter d` finds the index of letter d (4), then rotates the string right once, plus a number of times equal to that index, plus an additional time because the index was at least 4, for a total of 6 right rotations: decab.
